# 스키마 정의 후 Vector Store 생성 

---

---

# 1. Bedrock Client 생성

In [2]:
! pip list | grep langchain # 0.0.312
! pip list | grep opensearch # 2.3.2
! pip list | grep pypdf

langchain                                0.0.338
opensearch-py                            2.3.2
pypdf                                    3.17.1


In [3]:
%load_ext autoreload
%autoreload 2

import sys, os
# module_path = "../../../utils"
# sys.path.append(os.path.abspath(module_path))
# print(os.path.abspath(module_path))

module_path = "./utils"
sys.path.append(os.path.abspath(module_path))
print(os.path.abspath(module_path))

/root/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot/01_preprocess_docs/utils


In [4]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print(colored("\n== FM lists ==", "green"))
pprint(bedrock_info.get_list_fm_models())

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


# 2. Titan Embedding 모델 로딩

In [5]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

llm_emb = BedrockEmbeddings(client=boto3_bedrock)
llm_emb

BedrockEmbeddings(client=<botocore.client.BedrockRuntime object at 0x7f3b63325b70>, region_name=None, credentials_profile_name=None, model_id='amazon.titan-embed-text-v1', model_kwargs=None, endpoint_url=None)

# 3. Load all Json files

In [7]:
from utils.proc_docs import get_load_json, show_doc_json

In [8]:
import glob

# Specify the directory and file pattern for .txt files
folder_path = 'data/poc/preprocessed_json/all_processed_data.json'

# List all .txt files in the specified folder
json_files = glob.glob(folder_path)
# json_files = ['data/poc/customer_EFOTA.json']

# Load each item per json file and append to a list
doc_json_list = []
for file_path in json_files:
    doc_json = get_load_json(file_path)
    doc_json_list.append(doc_json)

print("all json files: ", len(doc_json_list))    
# Flatten the list of lists into a single list
all_docs = []
for item in doc_json_list:
        all_docs.extend(item)
        
print("all items: ", len(all_docs))

all json files:  1
all items:  1732


# 4. Chunking JSON Doc 

## Chunk Size and Chunk Overlap Size 결정

In [9]:
chunk_size = 1024
chunk_overlap = 256


## Chunking

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SpacyTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    separators=["\n\n", "\n", ".", " ", ""],
    length_function = len,
)

chunk_docs = text_splitter.split_documents(all_docs)
print(f"Number of chunk_docs after split and chunking= {len(chunk_docs)}")

Number of chunk_docs after split and chunking= 6825


In [10]:
chunk_docs[0:1]

[Document(page_content='AR demo barcodes. Use the following barcodes to test the Knox Capture AR features. Barcode 1 Barcode 2 Barcode 3 Barcode 4 Barcode 5 Barcode 6', metadata={'source': 'all_processed_data.json', 'seq_num': 1, 'title': 'AR demo barcodes', 'url': 'https://docs.samsungknox.com/admin/knox-capture/ar-demo-barcodes', 'project': 'KCAP', 'last_updated': '2023-10-16'})]

# 5. Index 생성

## Index 이름 결정

In [11]:
index_name = "genai-poc-knox-incremental-1024c-256o-v7"



## Index 스키마 정의

In [12]:
index_body = {
    'settings': {
        'index': {
            'knn': True,
            'knn.space_type': 'cosinesimil'  # Example space type
        }
    },
    'mappings': {
        'properties': {
            'metadata': {
                'properties': {
                               'source' : {'type': 'keyword'},
                               'last_updated': {'type': 'date'},
                               'project': {'type': 'keyword'},
                               'seq_num': {'type': 'long'},
                               'title': {'type': 'text'},  # For full-text search
                               'url': {'type': 'text'},  # For full-text search
                            }
            },            
            'text': {
                'type': 'text'
            },
            'vector_field': {
                'type': 'knn_vector',
                'dimension': 1536  # Replace with your vector dimension
            }
        }
    }
}


# 5. LangChain OpenSearch VectorStore 생성 
## 선수 조건


## 오픈 서치 도메인 및 인증 정보 세팅

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.

In [13]:
from utils.proc_docs import get_parameter

In [14]:
import boto3
ssm = boto3.client('ssm', 'us-east-1')

opensearch_domain_endpoint = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_domain_endpoint',
)

opensearch_user_id = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_userid',
)

opensearch_user_password = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_password',
)


In [15]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

## OpenSearch Client 생성

In [16]:
from utils.opensearch import opensearch_utils

In [17]:
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 오픈 서치 인덱스 생성 
- 오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [18]:
from utils.opensearch import opensearch_utils

In [19]:

index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    index_name
)

if index_exists:
    opensearch_utils.delete_index(
        os_client,
        index_name
    )

opensearch_utils.create_index(os_client, index_name, index_body)
index_info = os_client.indices.get(index=index_name)
print("Index is created")
pprint(index_info)

index_name=genai-poc-knox-incremental-1024c-256o-v7, exists=True

Deleting index:
{'acknowledged': True}

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'genai-poc-knox-incremental-1024c-256o-v7'}
Index is created
{'genai-poc-knox-incremental-1024c-256o-v7': {'aliases': {},
                                              'mappings': {'properties': {'metadata': {'properties': {'last_updated': {'type': 'date'},
                                                                                                      'project': {'type': 'keyword'},
                                                                                                      'seq_num': {'type': 'long'},
                                                                                                      'source': {'type': 'keyword'},
                                                                                                      'title': {'type': 'text'},
                                

## 랭체인 인덱스 연결 오브젝트 생성

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

In [20]:
from langchain.vectorstores import OpenSearchVectorSearch

In [21]:
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2",
    bulk_size=100000,
    timeout=60    
)
vector_db

## OpenSearch 에 문서 삽입

In [22]:
%%time

vector_db.add_documents(documents = chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000)


CPU times: user 18.6 s, sys: 1.06 s, total: 19.7 s
Wall time: 11min 8s


['c3ff24f0-9882-4490-8f05-f852f72df3b1',
 '4af97379-e466-4d42-bfca-58fb9c370a06',
 '3573bbec-0a2d-4a0d-b265-2afb1569d34e',
 '1af329d7-7094-41ca-8b7e-7b0cdb2f5355',
 'dbf7848d-866c-431c-8820-b7b647a66c85',
 'ed2aa035-b54e-4588-83b8-c41356664c2e',
 '58d57f66-3f75-458a-8c91-a6e20d3ba9c0',
 'e98f1f9e-a519-451b-b8a3-da69606496db',
 'a150da43-9507-442c-8d2c-43d07cd96030',
 '34ec4709-24df-46d8-832e-401be9dc2323',
 'b05aee10-b084-4415-9499-037b9a6cc509',
 '931332b6-0204-41bd-9a34-ab19de731d71',
 'aaa76e7c-964a-4eec-8ae7-e5787d24d7c5',
 '0192940a-febd-46e6-90b7-115c08a21d39',
 '9a663776-c942-4a84-935f-2b5f7f6fe7a8',
 '9ee1ab5e-095d-4a71-9086-dadab647fade',
 '0bdac915-5a7f-4989-9023-d281cdb70379',
 '63aa4e61-f063-4462-bde9-4692873de41d',
 '3692ecc4-40b2-47fa-8123-94b48cc6bedb',
 'b5eace65-56c8-4cf4-a59d-7d5c3857fbeb',
 'a7467b9d-8c46-4f00-af7d-827ebae2fcec',
 '3c777591-ef90-4596-ab54-de4d1280b6ed',
 'ee019ba7-cf00-4368-8c24-dcab73a29478',
 '16dea0aa-6606-4b4b-8a22-9620624c5817',
 'c535b851-2586-

# 6. 검색 테스트

## Lexical 검색

In [29]:
query = "how to add image"
 #query = "how to add image"
query = opensearch_utils.get_query(
    query=query
)

print("query: ", query)
response = opensearch_utils.search_document(os_client, query, index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

query:  {'query': {'bool': {'must': [{'match': {'text': {'query': 'how to add image', 'minimum_should_match': '0%', 'operator': 'or'}}}], 'filter': []}}}
# of searched docs:  10
# of display: 3
---------------------
_id in index:  14411a75-94b4-4541-87fb-05bc92741baa
10.037398
. Values Allow users to use Parallels desktop &mdash; Enables Parallels Desktop. When set, you must accept the end-user license agreement. Do not allow users to use Parallels desktop &mdash; Disables Parallels Desktop. Chrome OS 99 and higher Parallels Desktop Windows image The policy set for configuring the Windows OS image that the device user downloads on their Chromebooks in order to use Parallels Desktop. Chrome OS 99 and higher &gt; URL Specifies the address for the Windows image. Values Enter the URL. Chrome OS 99 and higher &gt; SHA-256 hash Specifies the SHA-256 hash of the Windows image. Values Enter the hash. Chrome OS 99 and higher Required disk space Specifies the free disk space required for Paralle

## 시멘틱 검색

In [30]:
vector_db.similarity_search("How to add image")

[Document(page_content='. You can add up to 10 image files in the PNG, JPG, JPEG, or GIF format (animated files are not supported). Each image file must be less than 5 MB. To upload an image file, click Add and select a file. To delete an image file, click next to the name of the uploaded image file. Note The device control command must be transferred to the device to apply an image file to it. &gt;&gt;&gt; Video Select a video file for the screen saver. You can add only one video file in the MP4 or MKV format. The video file must be less than 50 MB. To upload a video file, click Add and select a file. To delete a video file, click next to the name of the uploaded video file. Note The device control command must be transferred to the device to apply a video to it. &gt; Session timeout Allows the use of the session timeout feature for the Kiosk Browser', metadata={'source': 'all_processed_data.json', 'seq_num': 911, 'title': 'Android Enterprise policies', 'url': 'https://docs.samsungkno

## 하이브리드 검색

In [31]:
from langchain.chains.question_answering import load_qa_chain
from utils.rag import get_semantic_similar_docs, get_lexical_similar_docs, get_ensemble_results

In [32]:
import copy
from langchain.schema import Document
from langchain import PromptTemplate
from operator import itemgetter

In [33]:
from utils.proc_docs import search_hybrid

In [34]:
%%time


filter01 = "[]"
filter02 = "[]"

# query = "how to add image"
query = "How to add image"

search_hybrid_result = search_hybrid(
    query=query,
    vector_db=vector_db,
    k=3,
    index_name= index_name,
    os_client=os_client,
    filter=[
        {"term": {"metadata.type": filter01}},
        {"term": {"metadata.source": filter02}},
    ],
    Semantic_Search = False,    
    Lexical_Search = False,    
    Hybrid_Search = True,     
    minimum_should_match = 75,   
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"]
    ensemble_weights=[.5, .5], # 시멘트 서치에 가중치 0.5 , 키워드 서치 가중치 0.5 부여.
    verbose=True
)



Query: 
 How to add image
##############################
similar_docs_semantic
##############################

Score: 1.0
['. You can add up to 10 image files in the PNG, JPG, JPEG, or GIF format (animated files are not supported). Each image file must be less than 5 MB. To upload an image file, click Add and select a file. To delete an image file, click next to the name of the uploaded image file. Note The device control command must be transferred to the device to apply an image file to it. &gt;&gt;&gt; Video Select a video file for the screen saver. You can add only one video file in the MP4 or MKV format. The video file must be less than 50 MB. To upload a video file, click Add and select a file. To delete a video file, click next to the name of the uploaded video file. Note The device control command must be transferred to the device to apply a video to it. &gt; Session timeout Allows the use of the session timeout feature for the Kiosk Browser']
{'source': 'all_processed_data.jso

# 7. 검증 인덱스 생성

## Index 이름 결정

In [35]:
eval_index_name = "genai-poc-knox-en-eval-sample-0.02-1024c-256o-v10"

## Sampling

In [36]:
import random
def get_sampling_doc(seed, ratio, docs):

    random.seed(seed)
    
    eval_docs = docs[:int(len(docs)*ratio)]
    
    return eval_docs
    
eval_docs = get_sampling_doc(seed=200, ratio=0.02, docs= all_docs)
print("eval docs: ", len(eval_docs))
eval_docs[0:2]
    
    

eval docs:  34


[Document(page_content='AR demo barcodes. Use the following barcodes to test the Knox Capture AR features. Barcode 1 Barcode 2 Barcode 3 Barcode 4 Barcode 5 Barcode 6', metadata={'source': 'all_processed_data.json', 'seq_num': 1, 'title': 'AR demo barcodes', 'url': 'https://docs.samsungknox.com/admin/knox-capture/ar-demo-barcodes', 'project': 'KCAP', 'last_updated': '2023-10-16'}),
 Document(page_content="Videos. This section contains product and how-to videos related to Knox Capture. Getting started with Samsung Knox Capture In this video, we'll show you how to use Samsung Knox Capture to transform your mobile devices into powerful barcode scanners that can read, process, and output barcode data into other applications. Samsung Knox Capture: Enterprise-grade mobile scanning solution in Galaxy device This product intro video shows you how easy it is to transform rugged Samsung devices like the Galaxy XCover Pro into enterprise-grade barcode scanners.", metadata={'source': 'all_processe

In [37]:
chunk_docs = text_splitter.split_documents(eval_docs)
print(f"Number of chunk_docs after split and chunking= {len(chunk_docs)}")

Number of chunk_docs after split and chunking= 152


## 오픈 서치 인덱스 유무에 따라 삭제
오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [38]:
index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    eval_index_name
)

if index_exists:
    opensearch_utils.delete_index(
        os_client,
        eval_index_name
    )
    
opensearch_utils.create_index(os_client, eval_index_name, index_body)
index_info = os_client.indices.get(index=eval_index_name)
print("Index is created")
pprint(index_info)    

index_name=genai-poc-knox-en-eval-sample-0.02-1024c-256o-v10, exists=False

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'genai-poc-knox-en-eval-sample-0.02-1024c-256o-v10'}
Index is created
{'genai-poc-knox-en-eval-sample-0.02-1024c-256o-v10': {'aliases': {},
                                                       'mappings': {'properties': {'metadata': {'properties': {'last_updated': {'type': 'date'},
                                                                                                               'project': {'type': 'keyword'},
                                                                                                               'seq_num': {'type': 'long'},
                                                                                                               'source': {'type': 'keyword'},
                                                                                                               'title': {'type': 'text'},

## 검증 인덱스 생성

In [39]:
eval_vector_db = OpenSearchVectorSearch(
    index_name= eval_index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2",
    bulk_size=100000,
    timeout=60    
)
vector_db

In [40]:
%%time

eval_vector_db.add_documents(documents = chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000)


CPU times: user 428 ms, sys: 16.5 ms, total: 444 ms
Wall time: 14.9 s


['de4fb031-92be-4ddb-a5e4-274a27fd49df',
 '238ca909-e4d2-47d4-aea3-6578a66f7760',
 '30620de8-09b5-4d4e-9fc2-9cbd31a8e1a8',
 '682a4207-7aa1-4a81-a727-76c14a88dcea',
 '47b35eaa-9e03-40a0-9a88-918a15e355a3',
 '26ec5b0f-49dd-4988-9b3c-b5d6bbc4b42c',
 '1a911dcd-5e36-4b96-974d-9d85e556c8e0',
 'f06c3777-911b-4dc0-a2e4-9b44398effe4',
 '9b4499e8-8ffe-4dd7-94f2-cf9a81e1539e',
 '9a2de8f6-7474-408a-b006-a15a2e41a884',
 '540517a7-7d02-45fe-8ece-778d85909d9d',
 '183a7781-ba75-4f07-8129-8a9bceb950ea',
 'fdecb30a-6e39-4421-bc38-35d91b772ab2',
 '05cef790-bb2e-4450-b65c-c21642585406',
 'd97e9080-b229-4b74-b737-85404d03617a',
 '5097d86a-304c-40d8-a49f-f057c1f8232f',
 'a8f14bc6-6f8b-4e78-bcf0-034f953681f6',
 '2bc27b5e-0b1c-4841-82ff-0caa54abc6c9',
 '70a3b623-b27d-4123-b8d8-d2e18e635fd2',
 'e90a0e8f-c32f-4b8b-aa0a-48e5895da947',
 '92d7a4e6-e05d-4af7-b7bd-d3813fb94481',
 'f233df24-38d0-462f-bfa8-39e76df89729',
 'ae12eaab-b90d-4730-96aa-2af9d366298e',
 '6564cc91-f270-4dea-b3df-79015498a2f0',
 '1494bb28-afd4-

In [42]:
%%time


filter01 = "[]"
filter02 = "[]"

query = "how to add image"


search_hybrid_result = search_hybrid(
    query=query,
    vector_db= eval_vector_db,
    k=3,
    index_name= eval_index_name,
    os_client=os_client,
    filter=[
        {"term": {"metadata.type": filter01}},
        {"term": {"metadata.source": filter02}},
    ],
    Semantic_Search = False,    
    Lexical_Search = False,    
    Hybrid_Search = True,     
    minimum_should_match = 75,   
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"]
    ensemble_weights=[.5, .5], # 시멘트 서치에 가중치 0.5 , 키워드 서치 가중치 0.5 부여.
    verbose=True
)



Query: 
 how to add image
##############################
similar_docs_semantic
##############################

Score: 1.0
['. Note that this step affects the image in the product details. The thumbnail image always appears in the pop-up, regardless of the size you select. 4. In Fields, select the information that appears when the user the > button to expand the product details in the pop-up. For View-and-update AR templates, admins can specify which fields can be updated by the user by tapping the edit icon (pencil) next to a field name. The edit icon turns blue to indicate fields that can be overwritten. Note that fields only display the edit icon if they were given WRITE permissions in the database. 5. Tap Done to save the settings. Connecting AR to a database server For customers who want to display important product details in the AR overlay every time a barcode is scanned, a connection between Knox Capture and the database server must first be set up. Enterprise organizations are 

# A. Reference

- [Building a RAG AI with OpenSearch Serverless and LangChain](https://caylent.com/blog/building-a-rag-with-open-search-serverless-and-lang-chain)